#### Convert the Tensorflow model into tensorflow light model
After we build the model using TensorFlow, we will soon notice that the file size is too large and not optimized for deployment, especially on mobile or edge devices. This is where TensorFlow Lite (TFLite) comes into play. TFLite will help us convert the model to a more efficient format in .tflite. This will generate a small binary-sized model that is lightweight, low-latency and having a minor impact on accuracy.

In [2]:
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.models import load_model

# load the best saved model from our last training
myModel = load_model('Models/model_4.h5')

# create a TFLiteConverter object from a TensorFlow Keras model 
converter = tf.lite.TFLiteConverter.from_keras_model(myModel)

# converts a Keras model based on instance variable
myModel_tflite = converter.convert()

# Save the model
tflite_model_file = Path('clothing_classifier.tflite')
tflite_model_file.write_bytes(myModel_tflite)

INFO:tensorflow:Assets written to: C:\Users\cohenniv\AppData\Local\Temp\tmp0yd9s9n8\assets


190053256

Once we converted the model into the tflite file format, we can use it using a TFLite interpreter to see how the model will perform in making a prediction before deploying it on a mobile or edge device.

In [3]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
import tensorflow.lite as tflite
# in AWS Lambda, we need to use this import below
# import tflite_runtime.interpreter as tflite

# Create an interpreter interface for any model in TFLite
interpreter = tflite.Interpreter(model_path='clothing_classifier.tflite')
interpreter.allocate_tensors()

# Get a list of input details from the model
input_details = interpreter.get_input_details()
input_index = input_details[0]['index']

# Get a list of output details from the model
output_details = interpreter.get_output_details()
output_index = output_details[0]['index']

labels = ['dress','hat','longsleeve','outwear','pants','shirt','shoes','short','skirt','t-shirt']

# Obtain the image location
img_loc = 'datasets/test/hat/2a12baab-f020-42e3-8e6b-5d82e3ed0b55.jpg'

# load the image using load_img module
img = load_img(path=img_loc, target_size=(150,150))

# Turn the image into a 4D-array
X = np.expand_dims(img, axis=0)

# Normalize the image
X = X/255.0

# Turn the image into a Numpy array with float32 data type
X = X.astype('float32')

In [4]:
# set the value of the input tensor
interpreter.set_tensor(input_index, X)
interpreter.invoke()

# Get the value of the output tensor
preds = interpreter.get_tensor(output_index)
dict(zip(labels, preds[0]))

{'dress': 2.938265e-39,
 'hat': 1.0,
 'longsleeve': 5.7889116e-28,
 'outwear': 6.6241504e-34,
 'pants': 2.938265e-39,
 'shirt': 7.830514e-32,
 'shoes': 1.0016733e-28,
 'short': 2.938265e-39,
 'skirt': 1.075138e-37,
 't-shirt': 2.7841611e-34}